# Step 1: Prepare documents

In [1]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

As we can see the docs_raw structure are not in the same level and we need to iterate over elements in the docs_raw and elements under the document key, we can fixed using a loop

In [2]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

# Step 2: Create Embeddings using Pre-trained Models

In [17]:
#!pip install sentence_transformers==2.7.0

In [ ]:
# if you get an error do the following:
#!pip uninstall numpy 
#!pip uninstall torch
#!pip install numpy==1.26.4
#!pip install torch

In [4]:
from sentence_transformers import SentenceTransformer

In [5]:
model = SentenceTransformer("all-mpnet-base-v2")

In [6]:
len(model.encode("This is a simple sentence"))

768

In [7]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

The one that has the most information is either the text field or the question field, so it makes sense to use the embeddings on this portion of the document, for that we will be using a for loop and get the model.encode into each of the text in the documents file

In [8]:
operations = []
for doc in documents:
    doc['text_vector'] = model.encode(doc['text']).tolist()
    operations.append(doc)

In [9]:
operations[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.03570346161723137,
  -0.06891428679227829,
  -0.04448342323303223,
  0.006250153295695782,
  -0.042247094213962555,
  -0.00645399559289217,
  0.02754814364016056,
  -0.01861027255654335,
  -0.05203750357031822,
  -0.0024047393817454576,
  0.05745450034737587,
  -0.015592600218951702,
  0.015488838776946068,
  0.0987016037106514,
  0.01564955711364746,
  -0.061

## Step 3: Setup ElasticSearch connection

Before setting up the elasticsearch we need to run the docker container

```shell
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

In [10]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

es_client.info()

ObjectApiResponse({'name': '42296c5b4233', 'cluster_name': 'docker-cluster', 'cluster_uuid': '9XkmLjroQ56StdjHQ0UdFQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Step 4: Create Mappings and Index

- Mapping is the process of defining how a document, and the fields it contains, are stored and indexed.
- Each document is a collection of fields, which each have their own data type
- We can compare mapping to a database schema in how it describes the fields and properties that documents hold, the datatype of each field (e.g., string, integer, or date), and how those fields should be indexed and stored

In [11]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [12]:
index_name = "course-questions"

# This delete allow to create a new index by changing the settings
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

## Step 5: Add documents into index

In [13]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

## Step 6: Create end user query

In [14]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [15]:
query = {
    "field" : "text_vector",
    "query_vector": vector_search_term,
    "k":5, # five nearest neighboors
    "num_candidates":10000,
}

In [16]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'YzOg55ABmf5dQ79g1604',
  '_score': 0.7147919,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'djOh55ABmf5dQ79gFbAE',
  '_score': 0.61347336,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares a

In [17]:
# Note: I made a minor modification to the query shown in the notebook here
# (compare to the one shown in the video)
# Included "knn" in the search query (to perform a semantic search) along with the filter  
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [20]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"},
    },
    knn=knn_query,
    size=5,
    explain=True
)

In [21]:
response["hits"]["hits"]

[{'_shard': '[course-questions][0]',
  '_node': '7lspun-lR7mvm_cLVyUqMA',
  '_index': 'course-questions',
  '_id': 'YzOg55ABmf5dQ79g1604',
  '_score': 11.614713,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965461671352386,
    -0.000626126304268837,
    -0.01662949100136757,
    0.05285150930285454,
    0.05476527288556099,
    -0.03133990615606308,
    0.029942581430077553,
    -0.04808562621474266,
    0.04467551037669182,
    0.005839474033564329,
    0.016233040019869804,
    0.012001154012978077,
    -0.031222281977534294,
    0.016600528731942177,
    -0.04886901378631592,
    -0.06496307998895645,
    0.046434223651885986,
    -0.009297756478190422,
    -0.0642528235912323,
    -0.01373267825692892,


## Step 7: Perform Keyword search with Semantic Search (Hybrid/Advance search)

When we use the user input it is called keyword search, to have a semantic search we need to pass in the user input as embeddings

In [ ]:
response = es_client.search(
    index=index_name,
    query={
        "bool": {
            "must": {
                "multi_match":
                            {"query": "windows or python?",
                             "fields": ["text", "question", "course", "title"],
                             "type": "best_fields"
                            }
                        },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
)